In [ ]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem
from sympy import Rational
import dill

dill.settings["recurse"] = True
from sympy import Rational as R

### Get basis functions

In [ ]:
argyris_basis = symfem.create_element("triangle", "Argyris", 5)
argyris_basis = argyris_basis.get_basis_functions()

# Necessary symbols

In [ ]:
l1, l2, l3 = sympy.symbols("l1:4")
x, y, x_1, y_1, x_2, y_2, x_3, y_3 = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")
n1, n2, n3 = np.array(sympy.symbols("nx_1:4 ny_1:4")).reshape(2, 3).T
t1, t2, t3 = np.array(sympy.symbols("tx_1:4 ty_1:4")).reshape(2, 3).T
tau_1 = sympy.Matrix([t1[0] ** 2, 2 * t1[0] * t1[1], t1[1] ** 2])
tau_2 = sympy.Matrix([t2[0] ** 2, 2 * t2[0] * t2[1], t2[1] ** 2])
tau_3 = sympy.Matrix([t3[0] ** 2, 2 * t3[0] * t3[1], t3[1] ** 2])
J = sympy.Matrix(np.array(sympy.symbols("J(1:3)1:3")).reshape(2, 2).tolist())
THETA = sympy.Matrix(np.array(sympy.symbols("H(1:4)1:4")).reshape(3, 3).tolist())
B1 = sympy.Matrix(np.array(sympy.symbols("BI_(1:3)1:3")).reshape(2, 2).tolist())
B2 = sympy.Matrix(np.array(sympy.symbols("BII_(1:3)1:3")).reshape(2, 2).tolist())
B3 = sympy.Matrix(np.array(sympy.symbols("BIII_(1:3)1:3")).reshape(2, 2).tolist())
right_vals = sympy.Matrix(sympy.symbols("\phi_1:22"))

syms = [
    *n1,
    *n2,
    *n3,
    *t1,
    *t2,
    *t3,
    l1,
    l2,
    l3,
    *np.array(J.tolist()).flatten(),
    *np.array(THETA.tolist()).flatten(),
    *np.array(B1.tolist()).flatten(),
    *np.array(B2.tolist()).flatten(),
    *np.array(B3.tolist()).flatten(),
    *right_vals,
]

### Calculate bilinear form and right values for forward pushed reference element

In [ ]:
N = len(argyris_basis)
result_b = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]
result_A = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = argyris_basis[idx]
    second = argyris_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(x) * J.row(0)[0] + first.diff(y) * J.row(1)[0]
    u_y = first.diff(x) * J.row(0)[1] + first.diff(y) * J.row(1)[1]

    u_xx = u_x.diff(x) * J.row(0)[0] + u_x.diff(y) * J.row(1)[0]
    u_xy = u_x.diff(x) * J.row(0)[1] + u_x.diff(y) * J.row(1)[1]

    u_yy = u_y.diff(x) * J.row(0)[1] + u_y.diff(y) * J.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(x) * J.row(0)[0] + second.diff(y) * J.row(1)[0]
    v_y = second.diff(x) * J.row(0)[1] + second.diff(y) * J.row(1)[1]

    v_xx = v_x.diff(x) * J.row(0)[0] + v_x.diff(y) * J.row(1)[0]
    v_xy = v_x.diff(x) * J.row(0)[1] + v_x.diff(y) * J.row(1)[1]

    v_yy = v_y.diff(x) * J.row(0)[1] + v_y.diff(y) * J.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------bilinear-form------------------------------------------------------------------
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy
    # ------------------------------------------------------------------------------------------------------

    # -----------------------integrals----------------------------------------------------------------------
    sym_int_B = sympy.integrate(A + 2 * B + C, (y, 0, 1 - x), (x, 0, 1))
    sym_int_b = sympy.integrate(A + 2 * b + C, (y, 0, 1 - x), (x, 0, 1))
    sym_int_A = sympy.integrate(first * second, (y, 0, 1 - x), (x, 0, 1))

    result_B[idx][jdx] = sym_int_B
    result_b[idx][jdx] = sym_int_b
    result_A[idx][jdx] = sym_int_A
# -----------------------------------------------------------------------------------------------------

result_b = sympy.Matrix(result_b)
result_B = sympy.Matrix(result_B)
result_A = sympy.Matrix(result_A)

result_A = result_A * abs(J.inv().det())
result_b = result_b * abs(J.inv().det())
result_B = result_B * abs(J.inv().det())

In [ ]:
result_C = [0 for j in range(N)]

for idx in tqdm(range(N)):
    u = argyris_basis[idx]
    sym_int_C = sympy.integrate(u, (y, 0, 1 - x), (x, 0, 1))
    result_C[idx] = sym_int_C

result_C = sympy.Matrix(result_C)
result_C = result_C * abs(J.inv().det())

In [ ]:
result_A = sympy.simplify(str(result_A))

In [ ]:
result_b = sympy.simplify(str(result_b))

In [ ]:
result_B = sympy.simplify(str(result_B))

In [ ]:
result_C = sympy.simplify(str(result_C))

### Save the results

In [ ]:
dill.dump(result_b, open("../calculations/result_b", "wb"))
dill.dump(result_B, open("../calculations/result_B", "wb"))
dill.dump(result_A, open("../calculations/result_A", "wb"))

### Load the results

In [ ]:
result_b = dill.load(open("../calculations/result_b", "rb"))
result_B = dill.load(open("../calculations/result_B", "rb"))
result_A = dill.load(open("../calculations/result_A", "rb"))

# Define transformation matrices

### $V^{c}$

In [ ]:
V_c = sympy.diag(
    1,
    J.inv().T,
    THETA.inv(),
    1,
    J.inv().T,
    THETA.inv(),
    1,
    J.inv().T,
    THETA.inv(),
    B1,
    B2,
    B3,
)

### $E$

In [ ]:
E = np.zeros((21, 24), dtype=int)
for i in range(21):
    for j in range(24):
        if (i <= 18) and (j <= 18) and (i == j):
            E[i][j] = 1
        elif (i == 19) and (j == 20):
            E[i][j] = 1
        elif (i == 20) and (j == 22):
            E[i][j] = 1
E = sympy.Matrix(E)

### $D$

In [ ]:
D = sympy.Matrix(
    [
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [
            0,
            0,
            0,
            0,
            0,
            0,
            -R(15, 8) / l1,
            -R(7, 16) * t1[0],
            -R(7, 16) * t1[1],
            -l1 / 32 * tau_1[0],
            -l1 / 32 * tau_1[1],
            -l1 / 32 * tau_1[2],
            +R(15, 8) / l1,
            -R(7, 16) * t1[0],
            -R(7, 16) * t1[1],
            +l1 / 32 * tau_1[0],
            +l1 / 32 * tau_1[1],
            +l1 / 32 * tau_1[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [
            -R(15, 8) / l2,
            -R(7, 16) * t2[0],
            -R(7, 16) * t2[1],
            -l2 / 32 * tau_2[0],
            -l2 / 32 * tau_2[1],
            -l2 / 32 * tau_2[2],
            0,
            0,
            0,
            0,
            0,
            0,
            +R(15, 8) / l2,
            -R(7, 16) * t2[0],
            -R(7, 16) * t2[1],
            +l2 / 32 * tau_2[0],
            +l2 / 32 * tau_2[1],
            +l2 / 32 * tau_2[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [
            -R(15, 8) / l3,
            -R(7, 16) * t3[0],
            -R(7, 16) * t3[1],
            -l3 / 32 * tau_3[0],
            -l3 / 32 * tau_3[1],
            -l3 / 32 * tau_3[2],
            +R(15, 8) / l3,
            -R(7, 16) * t3[0],
            -R(7, 16) * t3[1],
            +l3 / 32 * tau_3[0],
            +l3 / 32 * tau_3[1],
            +l3 / 32 * tau_3[2],
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
        ],
    ]
)

### $M$

In [ ]:
M = (E @ V_c @ D).T

### Apply transformations

In [ ]:
bilinear_form_biharmonic_without_mixed = M @ result_b @ M.T

In [ ]:
bilinear_form_biharmonic_with_mixed = M @ result_B @ M.T

In [ ]:
right_part = M @ result_A @ M.T @ right_vals

In [ ]:
integral_values = M @ result_C

In [ ]:
bilinear_form_biharmonic_without_mixed_f = sympy.lambdify(
    syms, bilinear_form_biharmonic_without_mixed
)

In [ ]:
bilinear_form_biharmonic_with_mixed_f = sympy.lambdify(
    syms, bilinear_form_biharmonic_with_mixed
)

In [ ]:
right_part_f = sympy.lambdify(syms, right_part)

In [ ]:
integral_values_f = sympy.lambdify(syms[:-21], integral_values)

### Save the results

In [ ]:
dill.dump(
    bilinear_form_biharmonic_without_mixed_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_without_mixed", "wb"),
)
dill.dump(
    bilinear_form_biharmonic_with_mixed_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_with_mixed", "wb"),
)
dill.dump(
    right_part_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_right_values", "wb"),
)
dill.dump(
    integral_values_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_integral_values", "wb"),
)